In [67]:
import numpy as np
import collections.abc
collections.Iterable = collections.abc.Iterable
from collections import OrderedDict
import pylogit as pl
import pandas as pd
import scipy.special
import json

In [68]:
df = pd.read_csv('history.csv')
df

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
0,3574,0,7,19,4,13,0,0,0,0,...,0,5,2,3,9,6,2,3,11,3
1,3574,1,18,12,17,14,0,0,0,0,...,0,13,2,3,3,13,2,3,3,4
2,3574,2,2,16,9,3,0,0,0,0,...,0,13,2,5,4,13,2,6,5,2
3,3574,3,10,20,1,6,0,0,0,0,...,0,13,10,2,13,13,13,2,13,3
4,3574,4,8,15,11,5,0,0,0,0,...,0,13,2,11,6,13,2,13,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17450,8763,0,1,15,14,11,4,2,0,0,...,-2,3,2,13,13,4,2,13,13,2
17451,8763,1,9,3,8,20,-1,3,2,0,...,-1,13,7,6,2,13,9,9,2,2
17452,8763,2,2,12,6,19,1,0,1,-2,...,-3,10,8,4,2,13,10,5,2,4
17453,8763,3,10,7,13,18,0,3,0,-2,...,-3,13,2,3,9,13,2,3,11,2


In [69]:
df.describe()

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
count,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,...,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000
mean,6895.217416,2.000000,10.594958,10.437353,10.527757,10.439931,0.970496,0.988026,0.969350,0.983157,...,-1.209625,7.924148,7.042051,5.981438,4.687310,8.248525,7.416786,6.361673,5.057405,2.747522
std,1124.586097,1.414254,5.748542,5.784008,5.782101,5.749531,1.732478,1.735760,1.733119,1.732663,...,0.970576,4.460331,4.421288,4.184334,3.634178,4.532492,4.556024,4.384231,3.926012,1.104911
min,3574.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-5.000000,-5.000000,-5.000000,-5.000000,...,-6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
25%,5962.000000,1.000000,6.000000,5.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,-2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000
50%,6918.000000,2.000000,11.000000,10.000000,11.000000,10.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,7.000000,6.000000,4.000000,3.000000,9.000000,7.000000,5.000000,3.000000,3.000000
75%,7891.000000,3.000000,16.000000,15.000000,16.000000,15.000000,2.000000,2.000000,2.000000,2.000000,...,0.000000,13.000000,13.000000,10.000000,6.000000,13.000000,13.000000,11.000000,7.000000,4.000000
max,8763.000000,4.000000,20.000000,20.000000,20.000000,20.000000,8.000000,8.000000,8.000000,9.000000,...,0.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,4.000000


In [70]:
# combine columns pirate1 through pirate4 into a single column containing a list of pirate choices
def convert_df(df_orig):
    df = df_orig.copy()
    df['pirates'] = df[['pirate1', 'pirate2', 'pirate3', 'pirate4']].values.tolist()
    df['fas'] = df[['fa1', 'fa2', 'fa3', 'fa4']].values.tolist()
    df['pfas'] = df[['pfa1', 'pfa2', 'pfa3', 'pfa4']].values.tolist()
    df['nfas'] = df[['nfa1', 'nfa2', 'nfa3', 'nfa4']].values.tolist()
    df['opening_odds'] = df[['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4']].values.tolist()
    df['closing_odds'] = df[['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']].values.tolist()
    df = df.drop(['pirate1', 'pirate2', 'pirate3', 'pirate4'], axis=1)
    df = df.drop(['fa1', 'fa2', 'fa3', 'fa4'], axis=1)
    df = df.drop(['pfa1', 'pfa2', 'pfa3', 'pfa4'], axis=1)
    df = df.drop(['nfa1', 'nfa2', 'nfa3', 'nfa4'], axis=1)
    df = df.drop(['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4'], axis=1)
    df = df.drop(['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4'], axis=1)
    df['match_id'] = df['round'] * 5 + df['arena']
    return df
df = convert_df(df)
df

,round,arena,winner,pirates,fas,pfas,nfas,opening_odds,closing_odds,match_id
0,3574,0,3,"[7, 19, 4, 13]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[5, 2, 3, 9]","[6, 2, 3, 11]",17870
1,3574,1,4,"[18, 12, 17, 14]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 3, 3]","[13, 2, 3, 3]",17871
2,3574,2,2,"[2, 16, 9, 3]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 5, 4]","[13, 2, 6, 5]",17872
3,3574,3,3,"[10, 20, 1, 6]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 10, 2, 13]","[13, 13, 2, 13]",17873
4,3574,4,2,"[8, 15, 11, 5]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 11, 6]","[13, 2, 13, 6]",17874
...,...,...,...,...,...,...,...,...,...,...
17450,8763,0,2,"[1, 15, 14, 11]","[4, 2, 0, 0]","[5, 3, 2, 2]","[-1, -1, -2, -2]","[3, 2, 13, 13]","[4, 2, 13, 13]",43815
17451,8763,1,2,"[9, 3, 8, 20]","[-1, 3, 2, 0]","[0, 4, 4, 1]","[-1, -1, -2, -1]","[13, 7, 6, 2]","[13, 9, 9, 2]",43816
17452,8763,2,4,"[2, 12, 6, 19]","[1, 0, 1, -2]","[3, 0, 2, 1]","[-2, 0, -1, -3]","[10, 8, 4, 2]","[13, 10, 5, 2]",43817
17453,8763,3,2,"[10, 7, 13, 18]","[0, 3, 0, -2]","[1, 3, 3, 1]","[-1, 0, -3, -3]","[13, 2, 3, 9]","[13, 2, 3, 11]",43818


In [71]:
# explode each row of df into four rows, so that the resulting df
# contains columns of round, arena, win (0 or 1), pirate, fa, opening_odds, closing_odds, match_id
def convert_to_long_format_with_win(df):
    long_format_columns = ['round', 'arena', 'win', 'pirate', 'fa', 'pfa', 'nfa', 'position', 'position_factor', 'is_pos1', 'is_pos2', 'is_pos3', 'is_pos4', 'opening_odds', 'closing_odds', 'match_id']
    # positional effect is significant but sorta non-linear; this is taken from a model data
    # and we'll calculate each pirates "suscetibility to positional effects" by fitting a multiplier
    position_factor = [-0.1856, 0, 0.2848, 0.5554]
    long_format = pd.DataFrame([
        (
            tup.round,
            tup.arena,
            1 if 'winner' in df.columns and tup.winner == (i + 1) else 0,
            tup.pirates[i],
            tup.fas[i],
            tup.pfas[i],
            tup.nfas[i],
            i + 1,
            position_factor[i],
            1 if i + 1 == 1 else 0,
            1 if i + 1 == 2 else 0,
            1 if i + 1 == 3 else 0,
            1 if i + 1 == 4 else 0,
            tup.opening_odds[i],
            tup.closing_odds[i],
            tup.match_id
        )
        for tup in df.itertuples() for i in range(4)
    ],columns=long_format_columns)
    long_format['log_opening_implied_winrate'] = np.log(1.0 / long_format['opening_odds'])
    return long_format

def convert_to_long_format_without_win(df):
    return convert_to_long_format_with_win(df).drop(['win'], axis=1)

long_format = convert_to_long_format_with_win(df)
long_format

,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,3574,0,0,7,0,0,0,1,-0.1856,1,0,0,0,5,6,17870,-1.609438
1,3574,0,0,19,0,0,0,2,0.0000,0,1,0,0,2,2,17870,-0.693147
2,3574,0,1,4,0,0,0,3,0.2848,0,0,1,0,3,3,17870,-1.098612
3,3574,0,0,13,0,0,0,4,0.5554,0,0,0,1,9,11,17870,-2.197225
4,3574,1,0,18,0,0,0,1,-0.1856,1,0,0,0,13,13,17871,-2.564949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69815,8763,3,0,18,-2,1,-3,4,0.5554,0,0,0,1,9,11,43818,-2.197225
69816,8763,4,0,17,5,6,-1,1,-0.1856,1,0,0,0,2,2,43819,-0.693147
69817,8763,4,0,5,1,1,0,2,0.0000,0,1,0,0,5,5,43819,-1.609438
69818,8763,4,0,16,1,2,-1,3,0.2848,0,0,1,0,4,4,43819,-1.386294


In [72]:
PIRATE_NAMES = {
    1: "Dan",
    2: "Sproggie",
    3: "Orvinn",
    4: "Lucky",
    5: "Edmund",
    6: "Peg Leg",
    7: "Bonnie",
    8: "Puffo",
    9: "Stuff",
    10: "Squire",
    11: "Crossblades",
    12: "Stripey",
    13: "Ned",
    14: "Fairfax",
    15: "Gooblah",
    16: "Franchisco",
    17: "Federismo",
    18: "Blackbeard",
    19: "Buck",
    20: "Tailhook",
}

In [73]:
pirates_including_the_first = list(range(1, 21))
pirates_except_the_first = list(range(2, 21))

In [74]:
POSITIVE_FAS = {
    1: {1: 2, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 2, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 2, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    4: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    8: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 1, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    12: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    13: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    14: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    16: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 2, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 2, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 2},
    18: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    19: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0}
}
NEGATIVE_FAS = {
    1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    8: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    12: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    13: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    14: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    16: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    18: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    19: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0}
}

def get_ongoing_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [75]:
# backtesting
def get_data_for_round(df, round):
    return df[df['round'] == round]

convert_to_long_format_with_win(get_data_for_round(df, 8641))


,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8641,0,0,19,2,2,0,1,-0.1856,1,0,0,0,4,4,43205,-1.386294
1,8641,0,0,12,4,5,-1,2,0.0000,0,1,0,0,13,13,43205,-2.564949
2,8641,0,0,2,3,5,-2,3,0.2848,0,0,1,0,13,13,43205,-2.564949
3,8641,0,1,15,3,4,-1,4,0.5554,0,0,0,1,2,2,43205,-0.693147
4,8641,1,0,10,1,1,0,1,-0.1856,1,0,0,0,13,13,43206,-2.564949
5,8641,1,0,20,-1,1,-2,2,0.0000,0,1,0,0,8,12,43206,-2.079442
6,8641,1,0,8,-2,1,-3,3,0.2848,0,0,1,0,13,13,43206,-2.564949
7,8641,1,1,4,3,3,0,4,0.5554,0,0,0,1,2,2,43206,-0.693147
8,8641,2,1,17,4,5,-1,1,-0.1856,1,0,0,0,2,2,43207,-0.693147
9,8641,2,0,7,-1,1,-2,2,0.0000,0,1,0,0,6,6,43207,-1.791759


In [76]:
def generate_bets():
    for i in range(5):
        for j in range(5):
            for k in range(5):
                for l in range(5):
                    for m in range(5):
                        if i != 0 or j != 0 or k != 0 or l != 0 or m != 0:
                            yield (i, j, k, l, m)

len(list(generate_bets()))

3124

In [77]:
def get_odds_of_bet(bet, odds):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= odds[arena * 4 + pos - 1]
    return prod

tmp = convert_to_long_format_with_win(get_data_for_round(df, 8641))
get_odds_of_bet((4,2,0,0,2), tmp['closing_odds'])

96

In [78]:
def get_estimated_probabilities(df, model):
    return model.predict(df)

# get_estimated_probabilities(tmp, with_position_mnl)

In [79]:
def get_estimated_probability_of_bet(bet, probabilities):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= probabilities[arena * 4 + pos - 1]
    return prod

# get_estimated_probability_of_bet((4,2,0,0,2), get_estimated_probabilities(tmp, with_position_mnl))

In [80]:
def get_estimated_return_per_unit_of_bet(bet, odds, probabilities):
    return get_odds_of_bet(bet, odds) * get_estimated_probability_of_bet(bet, probabilities)

# get_estimated_return_per_unit_of_bet((4,2,0,0,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl))

In [81]:
def get_max_possible_bet_amount_floor(odds):
    return 1000000 // odds

def get_max_possible_bet_amount_ceil(odds):
    return -(1000000 // -odds)

get_max_possible_bet_amount_floor(96)
get_max_possible_bet_amount_ceil(96)

10417

In [82]:
def get_estimated_return_of_bet_amount(bet, odds, probabilities, bet_amount):
    odds_of_bet = get_odds_of_bet(bet, odds)
    estimated_probability = get_estimated_probability_of_bet(bet, probabilities)
    payout = min(1000000, bet_amount * odds_of_bet)
    return payout * estimated_probability

def get_best_bet_amount_and_estimated_return(bet, odds, probabilities):
    max_possible_bet_amount_floor = get_max_possible_bet_amount_floor(get_odds_of_bet(bet, odds))
    max_possible_bet_amount_ceil = get_max_possible_bet_amount_ceil(get_odds_of_bet(bet, odds))
    estimated_return_bet_amount_floor = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_floor)
    estimated_return_bet_amount_ceil = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_ceil)
    if (estimated_return_bet_amount_floor - max_possible_bet_amount_floor) >= (estimated_return_bet_amount_ceil - max_possible_bet_amount_ceil):
        return max_possible_bet_amount_floor, estimated_return_bet_amount_floor
    else:
        return max_possible_bet_amount_ceil, estimated_return_bet_amount_ceil

def get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap):
    amount, estimated_return = get_best_bet_amount_and_estimated_return(bet, odds, probabilities)
    if cap >= amount:
        return amount, estimated_return
    else:
        return cap, get_estimated_return_of_bet_amount(bet, odds, probabilities, cap)


In [83]:
def get_profitable_bets(odds, probabilities, cap):
    for bet in generate_bets():
        if get_estimated_return_per_unit_of_bet(bet, odds, probabilities) > 1:
            bet_amount, estimated_return = get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap)
            estimated_profit = estimated_return - bet_amount
            return_if_won = min(1000000, bet_amount * get_odds_of_bet(bet, odds))
            yield bet, bet_amount, estimated_return, estimated_profit, return_if_won

# 10 best bets for round 8641
# sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)), key=lambda x: x[3], reverse=True)[:10]

In [84]:
def is_winning_bet(bet, winners):
    for arena, pos in enumerate(bet):
        if pos != 0 and winners[arena] != pos:
            return False
    return True

def get_actual_return_of_bet(bet, winners, return_if_won):
    if is_winning_bet(bet, winners):
        return min(1000000, return_if_won)
    else:
        return 0


In [85]:
history_rounds = df["round"].unique().tolist()
len(history_rounds)

3491

In [86]:
from tqdm import tqdm
def backtest_model_with_cap(mnl, cap, history_rounds):
    total_bet_amount = 0
    total_return = 0
    total_potential_bet_amount = cap * 10 * len(history_rounds)
    for round in tqdm(history_rounds):
        tmp = convert_to_long_format_with_win(get_data_for_round(df, round))
        winners = df[df['round'] == round]['winner'].tolist()
        best_ten_bets = sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, mnl), cap)), key=lambda x: x[3], reverse=True)[:10]
        for bet, bet_amount, estimated_return, estimated_profit, return_if_won in best_ten_bets:
            total_bet_amount += bet_amount
            total_return += get_actual_return_of_bet(bet, winners, return_if_won)
    return total_bet_amount, total_potential_bet_amount, total_return, total_return - total_bet_amount, 1 + (total_return - total_bet_amount) / total_potential_bet_amount

In [87]:
per_pirate_fa_and_per_pirate_pos_specification = OrderedDict()
per_pirate_fa_and_per_pirate_pos_names = OrderedDict()
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_per_pirate_pos_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_per_pirate_pos_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_per_pirate_pos_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["is_pos2"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["is_pos2"] = [f'is_pos2_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["is_pos3"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["is_pos3"] = [f'is_pos3_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_per_pirate_pos_specification["is_pos4"] = pirates_including_the_first
per_pirate_fa_and_per_pirate_pos_names["is_pos4"] = [f'is_pos4_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_per_pirate_pos_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_per_pirate_pos_specification,
model_type='MNL',
names=per_pirate_fa_and_per_pirate_pos_names)
per_pirate_fa_and_per_pirate_pos_mnl.fit_mle(np.zeros(119))
per_pirate_fa_and_per_pirate_pos_mnl.get_statsmodels_summary()

Log-likelihood at zero: -24,197.7681
Initial Log-likelihood: -24,197.7681


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 9.64 seconds.
Final log-likelihood: -18,713.2221


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               17,455
Model:             Multinomial Logit Model   Df Residuals:                   17,336
Method:                                MLE   Df Model:                          119
Date:                     Sun, 07 May 2023   Pseudo R-squ.:                   0.227
Time:                             11:57:09   Pseudo R-bar-squ.:               0.222
AIC:                            37,664.444   Log-Likelihood:            -18,713.222
BIC:                            38,588.763   LL-Null:                   -24,197.768
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
ASC_1_Dan                 -0.5506      0.155     -3.541      0.000      -0.855      -0.246
ASC_2_Sproggie            -2.3848      0.203    -11.771      0.000      -2.782      -1.988
ASC_3_Orvinn              -3.4786      0.244    -14.251      0.000      -3.957      -3.000
ASC_4_Lucky               -1.3890      0.160     -8.670      0.000      -1.703      -1.075
ASC_5_Edmund              -1.9177      0.163    -11.775      0.000      -2.237      -1.598
ASC_6_Peg Leg             -2.5675      0.185    -13.894      0.000      -2.930      -2.205
ASC_7_Bonnie              -2.3143      0.180    -12.861      0.000      -2.667      -1.962
ASC_8_Puffo               -2.8314      0.234    -12.089      0.000      -3.290      -2.372
ASC_9_Stuff               -3.9019      0.259    -15.056      0.000      -4.410      -3.394
ASC_10_Squire             -3.5882      0.256    -14.006      0.000      -4.090      -3.086
ASC_11_Crossblades        -3.1482      0.225    -13.997      0.000      -3.589      -2.707
ASC_12_Stripey            -2.1693      0.181    -12.010      0.000      -2.523      -1.815
ASC_13_Ned                -1.7063      0.176     -9.672      0.000      -2.052      -1.361
ASC_14_Fairfax            -2.5503      0.191    -13.328      0.000      -2.925      -2.175
ASC_16_Franchisco         -1.2579      0.163     -7.725      0.000      -1.577      -0.939
ASC_17_Federismo          -1.0598      0.159     -6.678      0.000      -1.371      -0.749
ASC_18_Blackbeard         -2.1826      0.189    -11.568      0.000      -2.552      -1.813
ASC_19_Buck               -0.5606      0.152     -3.691      0.000      -0.858      -0.263
ASC_20_Tailhook           -1.6608      0.159    -10.417      0.000      -1.973      -1.348
PFA_1_Dan                  0.1575      0.022      7.085      0.000       0.114       0.201
PFA_2_Sproggie             0.2631      0.035      7.541      0.000       0.195       0.331
PFA_3_Orvinn               0.2510      0.041      6.181      0.000       0.171       0.331
PFA_4_Lucky                0.1596      0.040      3.999      0.000       0.081       0.238
PFA_5_Edmund               0.2765      0.039      7.163      0.000       0.201       0.352
PFA_6_Peg Leg              0.3169      0.060      5.291      0.000       0.199       0.434
PFA_7_Bonnie               0.2477      0.036      6.848      0.000       0.177       0.319
PFA_8_Puffo                0.2858      0.039      7.319      0.000       0.209       0.362
PFA_9_Stuff                0.4114      0.076      5.405      0.000       0.262       0.561
PFA_10_Squire              0.1973      0.082      2.396      0.017       0.036       0.359
PFA_11_Crossblades         0.1735      0.046      3.750      0.000       0.083       0.264
PFA_12_Stripey             0.1990      0.033      6.019      0.000       0.134       0.264
PFA_13_Ned                 0.2165      0.038      5.686      0.000       0.142       0.291
PFA_14_Fair

In [88]:
def generate_javascript(params):
    sep = ",\n    "
    return f"""
export const LOGIT_INTERCEPTS = {{
    {sep.join([f"{i+1}: {params[i]}" for i in range(14)])}{sep}15: 0{sep}{sep.join([f"{i+2}: {params[i]}" for i in range(14, 19)])}
}};
export const LOGIT_PFA = {{
    {sep.join([f"{i-18}: {params[i]}" for i in range(19, 39)])}
}};
export const LOGIT_NFA = {{
    {sep.join([f"{i-38}: {params[i]}" for i in range(39, 59)])}
}};
export const LOGIT_IS_POS2 = {{
    {sep.join([f"{i-58}: {params[i]}" for i in range(59, 79)])}
}};
export const LOGIT_IS_POS3 = {{
    {sep.join([f"{i-78}: {params[i]}" for i in range(79, 99)])}
}};
export const LOGIT_IS_POS4 = {{
    {sep.join([f"{i-98}: {params[i]}" for i in range(99, 119)])}
}};
"""

print(generate_javascript(per_pirate_fa_and_per_pirate_pos_mnl.params))


export const LOGIT_INTERCEPTS = {
    1: -0.5505653467394124,
    2: -2.3848388387111976,
    3: -3.478558254027841,
    4: -1.3890053586244873,
    5: -1.9176565939575803,
    6: -2.5675152075793033,
    7: -2.3143353273249554,
    8: -2.8313558799919383,
    9: -3.9019335823968233,
    10: -3.5882258128035347,
    11: -3.148241571143587,
    12: -2.169326502336402,
    13: -1.7062936735036478,
    14: -2.5503454346078662,
    15: 0,
    16: -1.2578784592762349,
    17: -1.059757385133957,
    18: -2.1826351058662317,
    19: -0.5605783719468618,
    20: -1.6608180038196982
};
export const LOGIT_PFA = {
    1: 0.15751645987509694,
    2: 0.26306055273281875,
    3: 0.2510034096704227,
    4: 0.15957937973235922,
    5: 0.2765431062703744,
    6: 0.31686653297964323,
    7: 0.24768920967758712,
    8: 0.285786215512296,
    9: 0.41136162216849836,
    10: 0.19728776166082862,
    11: 0.1734956834280819,
    12: 0.1990091706829303,
    13: 0.21651930132706249,
    14: 0.246354673493688

In [89]:
def get_current_round_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'pfa1', 'pfa2', 'pfa3', 'pfa4', 'nfa1', 'nfa2', 'nfa3', 'nfa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            positive_fas = [0 for pirate in pirates]
            negative_fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            pfa1, pfa2, pfa3, pfa4 = positive_fas
            nfa1, nfa2, nfa3, nfa4 = negative_fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, pfa1, pfa2, pfa3, pfa4, nfa1, nfa2, nfa3, nfa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [90]:
def get_round(round_number):
    return convert_to_long_format_without_win(convert_df(get_current_round_df_from_file(f"raw_json/{round_number}.json")))
round_8643 = get_round(8643)
round_8643

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8643,0,2,-1,2,-3,1,-0.1856,1,0,0,0,13,13,43215,-2.564949
1,8643,0,16,2,2,0,2,0.0000,0,1,0,0,2,2,43215,-0.693147
2,8643,0,3,0,1,-1,3,0.2848,0,0,1,0,13,13,43215,-2.564949
3,8643,0,5,2,2,0,4,0.5554,0,0,0,1,2,2,43215,-0.693147
4,8643,1,9,-1,0,-1,1,-0.1856,1,0,0,0,13,13,43216,-2.564949
5,8643,1,18,1,2,-1,2,0.0000,0,1,0,0,13,13,43216,-2.564949
6,8643,1,1,4,4,0,3,0.2848,0,0,1,0,2,2,43216,-0.693147
7,8643,1,8,-1,2,-3,4,0.5554,0,0,0,1,13,13,43216,-2.564949
8,8643,2,19,0,1,-1,1,-0.1856,1,0,0,0,2,2,43217,-0.693147
9,8643,2,13,2,3,-1,2,0.0000,0,1,0,0,2,2,43217,-0.693147


In [91]:
per_pirate_fa_and_per_pirate_pos_mnl.predict(round_8643)

array([0.05727582, 0.37783439, 0.04717735, 0.51771244, 0.00945087,
       0.08582781, 0.86211004, 0.04261127, 0.48901203, 0.31520892,
       0.06205003, 0.13372901, 0.07904359, 0.32751036, 0.40627061,
       0.18717545, 0.04602858, 0.85870709, 0.0528648 , 0.04239953])

In [92]:
import string
def parse_bets(s):
    indices = [string.ascii_lowercase.find(c) for c in s]
    values = []
    for index in indices:
        values.append(index // 5)
        values.append(index % 5)

In [93]:
def get_best_10_bets(round, odds, model, cap):
    return sorted(list(get_profitable_bets(
        odds,
        get_estimated_probabilities(get_round(round), model), cap)),
        key=lambda x: x[3], reverse=True)[:10]

In [94]:
def get_bet_url(round, bets):
    bets_array = [v for bet in bets for v in bet[0]]
    # turn bets_array into chunks of 2 (pad 0 if necessary)
    if len(bets_array) % 2 == 1:
        bets_array.append(0)
    bets_chunks = [bets_array[i:i+2] for i in range(0, len(bets_array), 2)]
    # turn bets_chunks into a string
    bets_string = "".join([string.ascii_lowercase[5 * x + y] for x, y in bets_chunks])
    return f"/#round={round}&b={bets_string}"

In [95]:
from coloraide import Color, stop
def get_background_for_payout(payout):
    return Color.interpolate(['#50c17f', stop('transparent', np.exp(-1)), '#f76c6c'], space='srgb', premultiplied=True)(np.exp(-payout)).to_string()

get_background_for_payout(1.2)

'rgb(80 193 127 / 0.18127)'

In [96]:
def get_payouts(round, model, odds_override=None):
    round_df = get_round(round)
    estimated_probabilities = get_estimated_probabilities(round_df, model)
    estimated_probabilities_series = pd.Series(estimated_probabilities, name="estimated_probabilities")
    round_df = round_df.join(estimated_probabilities_series)
    if odds_override is not None:
        round_df["custom_odds"] = pd.Series(odds_override, name="custom_odds")
    else:
        round_df["custom_odds"] = round_df["closing_odds"]
    round_df["payouts"] = round_df.apply(lambda row: row["custom_odds"] * row["estimated_probabilities"], axis=1)
    return round_df.style.applymap(lambda x: f"background-color: {get_background_for_payout(x)}", subset=["payouts"])

In [97]:
current_round = 8766

In [98]:
manual_odds = get_round(current_round)['closing_odds'].tolist()
manual_odds

[13, 13, 4, 2, 13, 12, 2, 2, 2, 2, 13, 13, 7, 7, 2, 4, 13, 3, 2, 10]

In [99]:
per_pirate_fa_and_per_pirate_pos_mnl.predict(get_round(current_round))

array([0.03103153, 0.0551223 , 0.2718321 , 0.64201406, 0.01260969,
       0.0917667 , 0.42003575, 0.47558786, 0.47040604, 0.43566661,
       0.06056774, 0.03335961, 0.17494121, 0.13233104, 0.49546526,
       0.19726248, 0.01952026, 0.30708694, 0.59441538, 0.07897742])

In [34]:
# manual_odds = [5, 2, 13, 6, 13, 13, 2, 2, 6, 13, 4, 2, 4, 3, 3, 3, 2, 4, 9, 3]

In [100]:
get_payouts(current_round, per_pirate_fa_and_per_pirate_pos_mnl, odds_override=manual_odds)

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate,intercept,estimated_probabilities,custom_odds,payouts
0,8766,0,6,-2,0,-2,1,-0.185600,1,0,0,0,13,13,43830,-2.564949,1.000000,0.031032,13,0.403410
1,8766,0,18,1,5,-4,2,0.000000,0,1,0,0,13,13,43830,-2.564949,1.000000,0.055122,13,0.716590
2,8766,0,7,2,2,0,3,0.284800,0,0,1,0,3,4,43830,-1.098612,1.000000,0.271832,4,1.087328
3,8766,0,5,3,4,-1,4,0.555400,0,0,0,1,2,2,43830,-0.693147,1.000000,0.642014,2,1.284028
4,8766,1,10,-1,1,-2,1,-0.185600,1,0,0,0,13,13,43831,-2.564949,1.000000,0.012610,13,0.163926
5,8766,1,17,-1,2,-3,2,0.000000,0,1,0,0,12,12,43831,-2.484907,1.000000,0.091767,12,1.101200
6,8766,1,16,3,4,-1,3,0.284800,0,0,1,0,2,2,43831,-0.693147,1.000000,0.420036,2,0.840071
7,8766,1,20,1,1,0,4,0.555400,0,0,0,1,2,2,43831,-0.693147,1.000000,0.475588,2,0.951176
8,8766,2,15,0,1,-1,1,-0.185600,1,0,0,0,2,2,43832,-0.693147,1.000000,0.470406,2,0.940812
9,8766,2,19,1,2,-1,2,0.000000,0,1,0,0,2,2,43832,-0.693147,1.000000,0.435667,2,0.871333


In [39]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_per_pirate_pos_mnl, 1)

[((4, 0, 3, 2, 2), 1, 2.1339054883763677, 1.1339054883763677, 1078),
 ((4, 3, 3, 2, 2), 1, 2.0565834160310925, 1.0565834160310925, 2156),
 ((4, 0, 3, 2, 0), 1, 2.0441921390896285, 1.0441921390896285, 539),
 ((4, 3, 3, 2, 0), 1, 1.970120830248956, 0.9701208302489559, 1078),
 ((2, 0, 3, 2, 2), 1, 1.9367721008659888, 0.9367721008659888, 1078),
 ((2, 3, 3, 2, 2), 1, 1.8665931574614163, 0.8665931574614163, 2156),
 ((2, 0, 3, 2, 0), 1, 1.8553466052569925, 0.8553466052569925, 539),
 ((4, 4, 3, 2, 2), 1, 1.8537173093447987, 0.8537173093447987, 5390),
 ((0, 0, 3, 2, 2), 1, 1.8285296666766122, 0.8285296666766122, 154),
 ((3, 0, 3, 2, 2), 1, 1.815833100532072, 0.815833100532072, 308)]

In [40]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_per_pirate_pos_mnl, 1))

'/#round=8764&b=urosmureskkrmsmkrexmarndm'

In [41]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_per_pirate_pos_mnl, 11424)

[((0, 0, 3, 2, 0), 11424, 20010.905394835478, 8586.905394835478, 879648),
 ((4, 0, 3, 1, 0), 10204, 16935.810711899667, 6731.810711899667, 999992),
 ((4, 0, 3, 0, 2), 10204, 16754.499188871647, 6550.499188871647, 999992),
 ((4, 0, 3, 0, 0), 11424, 17969.075306193725, 6545.075306193725, 559776),
 ((4, 0, 0, 2, 0), 11424, 17326.288578596897, 5902.288578596897, 879648),
 ((0, 0, 3, 1, 2), 11424, 16960.319625932312, 5536.319625932312, 319872),
 ((3, 0, 3, 1, 2), 11424, 16842.554066046934, 5418.554066046934, 639744),
 ((0, 0, 3, 2, 2), 6493, 11872.643125731243, 5379.643125731243, 999922),
 ((4, 3, 3, 0, 0), 10204, 15468.533181148745, 5264.533181148745, 999992),
 ((2, 0, 3, 1, 0), 10204, 15371.255133380764, 5167.255133380764, 999992)]

In [42]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_per_pirate_pos_mnl, 11424))

'/#round=8764&b=aredfupodaucadhpqkdmxpcdf'